## Scraping

In [1]:
# Dependencies & Set-up
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


### NASA Mars News

In [2]:
# Set-up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Set URL
browser.visit('https://mars.nasa.gov/news/')

html = browser.html
soup = bs(html, 'html.parser')


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/mori/.wdm/drivers/chromedriver/mac64/88.0.4324.96]


In [3]:
# Search for news titles and teaser paragraphs
results = soup.find_all('ul', class_='item_list')

# Loop through results
for result in results:
    
    title = result.find_all('div', class_='content_title')
    paragraph = result.find_all('div', class_='article_teaser_body')
    
    # Extract the first title and paragraph, and assign to variables
    news_title = title[0].text
    news_paragraph = paragraph[0].text
    
    # Print results
    print(news_title)
    print(news_paragraph)

NASA's Perseverance Mars Rover Mission Honors Navajo Language
Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.


### JPL Mars Space Images - Featured Image

In [5]:
# Open browser to JPL Featured Image

# Set URL
browser.visit('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')

html = browser.html
soup = bs(html, 'html.parser')


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/mori/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [7]:
# Find image relative path
image = soup.find('a', class_='showimg')['href']
print(image)


image/featured/mars3.jpg


In [8]:
# Add relative path to full URL string
featured_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + image
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


### Mars Facts

In [ ]:
# 

### Mars Hemispheres

In [ ]:
# 


## MongoDB and Flask Application